# Tree Methods Consulting Project 

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.
___

**Think carefully about what this problem is really asking you to solve. While we will use Machine Learning to solve this, it won't be with your typical train/test split workflow. If this confuses you, skip ahead to the solution code along walk-through!**
____

In [2]:
import findspark
findspark.init('/home/stuthi/prep/spark-2.0.0-bin-hadoop2.7/')

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('trees').getOrCreate()

In [7]:
df = spark.read.csv('dog_food.csv',header=True, inferSchema=True)

In [8]:
df.describe().show()

+-------+------------------+------------------+------------------+------------------+-------------------+
|summary|                 A|                 B|                 C|                 D|            Spoiled|
+-------+------------------+------------------+------------------+------------------+-------------------+
|  count|               490|               490|               490|               490|                490|
|   mean|  5.53469387755102| 5.504081632653061| 9.126530612244897| 5.579591836734694| 0.2857142857142857|
| stddev|2.9515204234399057|2.8537966089662063|2.0555451971054275|2.8548369309982857|0.45221563164613465|
|    min|                 1|                 1|               5.0|                 1|                0.0|
|    max|                10|                10|              14.0|                10|                1.0|
+-------+------------------+------------------+------------------+------------------+-------------------+



In [10]:
df.head(2)

[Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0),
 Row(A=5, B=6, C=12.0, D=7, Spoiled=1.0)]

In [11]:
from pyspark.ml.feature import VectorAssembler

In [12]:
assm = VectorAssembler(inputCols=['A','B','C','D'], outputCol = 'features')

In [13]:
fdf = assm.transform(df)

In [14]:
fdf.head(2)

[Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0, features=DenseVector([4.0, 2.0, 12.0, 3.0])),
 Row(A=5, B=6, C=12.0, D=7, Spoiled=1.0, features=DenseVector([5.0, 6.0, 12.0, 7.0]))]

In [15]:
train, test = fdf.randomSplit([0.7,0.3])

In [16]:
from pyspark.ml.classification import RandomForestClassifier
rfc = RandomForestClassifier(labelCol='Spoiled',featuresCol='features')

In [17]:
rfc_model = rfc.fit(train.select(['Spoiled','features']))

In [23]:
preds = rfc_model.transform(test.select(['Spoiled','features']))

In [24]:
preds.head()

Row(Spoiled=1.0, features=DenseVector([1.0, 1.0, 10.0, 8.0]), rawPrediction=DenseVector([16.0, 4.0]), probability=DenseVector([0.8, 0.2]), prediction=0.0)

In [22]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [27]:
ev = MulticlassClassificationEvaluator(predictionCol='prediction',labelCol='Spoiled',metricName='accuracy')

In [28]:
ev.evaluate(preds)

0.9709302325581395

In [29]:
rfc_model.featureImportances

SparseVector(4, {0: 0.0081, 1: 0.0135, 2: 0.9565, 3: 0.0219})